In [18]:
%load_ext autoreload
%autoreload 2

import polars as pl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# CPI + CLI DATA COLLECTION


## OECD Composite Leading Indicator

In [ ]:
cli = obb.economy.composite_leading_indicator(start_date='2024-01-01', provider='oecd', country=['united_states'])
cli.to_polars()

## Consumer Price Index

### OECD Consumer Price Index

In [ ]:
oecd_cpi = obb.economy.cpi(start_date='2010-01-01', country='united_states', transform='yoy', provider='oecd',harmonized=False, expenditure='total')
oecd_cpi.results


### FRED Consumer Price Index

In [ ]:
fred_cpi = obb.economy.cpi(start_date='2024-07-01', country='united_states', transform='yoy', provider='fred')
fred_cpi.results

# humblCOMPASS 

In [21]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    # symbols=["AAPL"],
    # interval="1d",
    start_date="2020-01-01",
    end_date="2024-12-01",
    membership="humblPEON"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
compass =await toolbox.fundamental.humbl_compass(
    country="united_states",
    chart=True,
    recommendations=True
)



INFO: HumblCompassFetcher || START: fetch_data (async)
Couldn't retrieve {"country": "united_states", "end_date": "2024-12-01", "start_date": "2022-12-02", "z_score": null}, unexpected error
Traceback (most recent call last):
  File "/Users/jjfantini/github/humblFINANCE-org/humblDATA/.venv/lib/python3.12/site-packages/aiocache/decorators.py", line 157, in get_from_cache
    return await self.cache.get(key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jjfantini/github/humblFINANCE-org/humblDATA/.venv/lib/python3.12/site-packages/aiocache/base.py", line 64, in _enabled
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jjfantini/github/humblFINANCE-org/humblDATA/.venv/lib/python3.12/site-packages/aiocache/base.py", line 48, in _timeout
    return await asyncio.wait_for(func(self, *args, **kwargs), timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jjfantini/.local/share/uv/python/cpython-3.12.10

In [25]:
compass.to_polars()

date_month_start,country,cpi,cpi_3m_delta,cli,cli_3m_delta,humbl_regime
date,str,f64,f64,f64,f64,str
2022-12-01,"""United States""",6.45,-1.75,98.73,-0.3,"""humblBUST"""
2023-01-01,"""United States""",6.41,-1.34,98.69,-0.21,"""humblBUST"""
2023-02-01,"""United States""",6.04,-1.07,98.66,-0.14,"""humblBUST"""
2023-03-01,"""United States""",4.98,-1.47,98.64,-0.09,"""humblBUST"""
2023-04-01,"""United States""",4.93,-1.48,98.66,-0.03,"""humblBUST"""
…,…,…,…,…,…,…
2024-07-01,"""United States""",2.89,-0.46,99.74,0.06,"""humblBOOM"""
2024-08-01,"""United States""",2.53,-0.74,99.83,0.13,"""humblBOOM"""
2024-09-01,"""United States""",2.44,-0.53,99.96,0.26,"""humblBOOM"""


In [ ]:
compass.extra


In [ ]:
compass.show()

In [7]:
compass.warnings

[Warning_(category='HumblDataWarning', message='Start date adjusted to 2022-12-02 based on humblPEON membership (2Y of data).'),
 Warning_(category='HumblDataWarning', message='Z-score defaulted to None. No z-score data will be calculated.')]